<a href="https://colab.research.google.com/github/ayyucedemirbas/machine_learning_algorithms/blob/master/k_fold_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
(input_train, target_train), (input_test, target_test) = cifar10.load_data()
# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
input_train = input_train / 255
input_test = input_test / 255

# Merge inputs and targets
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

In [10]:
input_train.shape[1:]

(32, 32, 3)

In [15]:
inputs[1].shape

(32, 32, 3)

In [16]:
targets[1].shape

(1,)

In [3]:
import tensorflow
from tensorflow.keras import initializers

In [4]:
def get_model():
  model= Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3),kernel_initializer=initializers.GlorotUniform(seed=None)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu',kernel_initializer=initializers.GlorotUniform(seed=None)),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu', kernel_initializer=initializers.GlorotUniform(seed=None)),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(256, activation='relu', kernel_initializer=initializers.GlorotUniform(seed=None)),
        Dense(128, activation='relu', kernel_initializer=initializers.GlorotUniform(seed=None)),
        Dense(10, activation='softmax', kernel_initializer=initializers.GlorotUniform(seed=None))


      ])
  opt = tensorflow.keras.optimizers.Adam(learning_rate=0.005)
  acc = tensorflow.keras.metrics.SparseCategoricalAccuracy()
  mae = tensorflow.keras.metrics.MeanAbsoluteError()
  model.compile(optimizer= opt,
      loss = 'sparse_categorical_crossentropy', #sparse_categorical_crossentropy --> data labels are integers
      metrics= [acc, mae]
  )
  return model
  


In [5]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=4, shuffle=False)
input_shape = input_train.shape[1:]
# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold=[]
loss_per_fold=[]
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model= get_model()
  


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no}')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],epochs=8, batch_size=256, verbose=2)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])
  val_score=np.average(acc_per_fold)
  print("Average validation score")
  print(val_score)

  
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1
Epoch 1/8
176/176 - 4s - loss: 1.7177 - sparse_categorical_accuracy: 0.3556 - mean_absolute_error: 4.4231
Epoch 2/8
176/176 - 1s - loss: 1.3024 - sparse_categorical_accuracy: 0.5278 - mean_absolute_error: 4.4231
Epoch 3/8
176/176 - 1s - loss: 1.1231 - sparse_categorical_accuracy: 0.6005 - mean_absolute_error: 4.4231
Epoch 4/8
176/176 - 1s - loss: 1.0135 - sparse_categorical_accuracy: 0.6437 - mean_absolute_error: 4.4231
Epoch 5/8
176/176 - 1s - loss: 0.9438 - sparse_categorical_accuracy: 0.6674 - mean_absolute_error: 4.4231
Epoch 6/8
176/176 - 1s - loss: 0.8676 - sparse_categorical_accuracy: 0.6930 - mean_absolute_error: 4.4231
Epoch 7/8
176/176 - 1s - loss: 0.8075 - sparse_categorical_accuracy: 0.7147 - mean_absolute_error: 4.4231
Epoch 8/8
176/176 - 1s - loss: 0.7568 - sparse_categorical_accuracy: 0.7332 - mean_absolute_error: 4.4231
Score for fold 1: loss of 0.9678270220756531; sparse_catego

In [6]:
model=get_model()
history = model.fit(input_train, target_train,epochs=8, batch_size=256, verbose=2)

Epoch 1/8
196/196 - 2s - loss: 1.6985 - sparse_categorical_accuracy: 0.3643 - mean_absolute_error: 4.4200
Epoch 2/8
196/196 - 1s - loss: 1.2943 - sparse_categorical_accuracy: 0.5339 - mean_absolute_error: 4.4200
Epoch 3/8
196/196 - 1s - loss: 1.1270 - sparse_categorical_accuracy: 0.5994 - mean_absolute_error: 4.4200
Epoch 4/8
196/196 - 1s - loss: 1.0171 - sparse_categorical_accuracy: 0.6412 - mean_absolute_error: 4.4200
Epoch 5/8
196/196 - 1s - loss: 0.9343 - sparse_categorical_accuracy: 0.6725 - mean_absolute_error: 4.4200
Epoch 6/8
196/196 - 1s - loss: 0.8742 - sparse_categorical_accuracy: 0.6931 - mean_absolute_error: 4.4200
Epoch 7/8
196/196 - 1s - loss: 0.8275 - sparse_categorical_accuracy: 0.7095 - mean_absolute_error: 4.4200
Epoch 8/8
196/196 - 1s - loss: 0.7756 - sparse_categorical_accuracy: 0.7257 - mean_absolute_error: 4.4200


In [7]:
model.evaluate(input_test, target_test,batch_size=256)

40/40 [==============================] - 0s 7ms/step - loss: 1.0033 - sparse_categorical_accuracy: 0.6590 - mean_absolute_error: 4.4200


[1.003265380859375, 0.6589999794960022, 4.420001029968262]